In [ ]:
# !pip install flask --user
def func(dataset):
    import pandas as pd
#     dataset=pd.read_csv(requirement)
    CRE_corpus=dataset['feature']+','+dataset['benefit']+'.'
    from urllib.request import urlopen
    url="https://en.wikipedia.org/wiki/Category:Home_automation"
    data=urlopen(url)
    html=data.read()
    # print(html)
    data.close()
    from bs4 import BeautifulSoup as soup
    html_soup=soup(html,'html.parser')
    subcat=html_soup.findAll('div',{'id':'mw-subcategories'})
    list=subcat[0].findAll('li')
    HA_corpus=[]
    for i in range(0,len(list)):
        link=list[i].find('a')
        url1="https://en.wikipedia.org"+link['href']
        data1=urlopen(url1)
        html1=data1.read()
    #     print(html1)
        data1.close()
        html_soup1=soup(html1,'html.parser')
        subcat3=html_soup1.findAll('div',{'id':'mw-subcategories'})
        if len(subcat3)>0:
            list3=subcat3[0].findAll('li')
            for j in range(0,len(list3)):
                link3=list3[j].find('a')
                url3="https://en.wikipedia.org"+link3['href']
                data3=urlopen(url3)
                html3=data3.read()
                data3.close()
                html_soup3=soup(html3,'html.parser')
                cat3=html_soup3.findAll('div',{'id':'mw-pages'})
                list4=cat3[0].findAll('li')
                for k in range(0,len(list4)):
                    text4=list4[k].find('a').text
                    HA_corpus.append(text4)
        cat1=html_soup1.findAll('div',{'id':'mw-pages'})
        list1=cat1[0].findAll('li')
        for j in range(0,len(list1)):
            text1=list1[j].find('a').text
            HA_corpus.append(text1)
    # print(HA_corpus)   
    cat2=html_soup.findAll('div',{'id':'mw-pages'})
    list2=cat2[0].findAll('li')
    for i in range(0,len(list2)):
        text2=list2[i].find('a').text
        HA_corpus.append(text2)
    # print(HA_corpus)
    # 1. Tokenization
    import nltk
    nltk.download('punkt')
    HA_tokens=[]
    for i in range(0,len(HA_corpus)):
      alpha_tokens=[word for word in HA_corpus[i].split() if word.isalpha()]
      result=' '.join(alpha_tokens)
      tokens=nltk.word_tokenize(result)
      for j in range(0,len(tokens)):
        tokens[j]=tokens[j].lower()
      HA_tokens.append(tokens)
    # print(HA_tokens)
    CRE_tokens=[]
    for i in range(0,len(CRE_corpus)):
      alpha_tokens=[word for word in CRE_corpus[i].split() if word.isalpha()]
      result=' '.join(alpha_tokens)
      tokens=nltk.word_tokenize(result)
      for j in range(0,len(tokens)):
        tokens[j]=tokens[j].lower()
      CRE_tokens.append(tokens)
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    nltk.download('omw-1.4')
    from nltk.stem import WordNetLemmatizer
    nltk.download('wordnet')
    lemmatizer=WordNetLemmatizer()
    stop_words=set(stopwords.words('english'))
    new_HA_tokens=[]
    for i in range(0,len(HA_tokens)):
      temp=[]
      for token in HA_tokens[i]:
        if token not in stop_words:
          temp.append(lemmatizer.lemmatize(token))
      new_HA_tokens.append(temp)
    # print(new_HA_tokens)
    new_CRE_tokens=[]
    for i in range(0,len(CRE_tokens)):
      temp=[]
      for token in CRE_tokens[i]:
        if token not in stop_words:
          temp.append(lemmatizer.lemmatize(token))
      new_CRE_tokens.append(temp)
    # print(new_CRE_tokens)
    #pos_tagging
    from nltk.tag import pos_tag
    CRE=[]
    nltk.download('averaged_perceptron_tagger')
    for i in range(0,len(new_CRE_tokens)):
      CRE.append(pos_tag(new_CRE_tokens[i]))
    HA=[]
    for i in range(0,len(new_HA_tokens)):
      HA.append(pos_tag(new_HA_tokens[i]))
    # text chunking
    import nltk
    # nltk.download('maxent_ne_chunker')
    nltk.download('words')
    from nltk import RegexpParser
    grammer = "NP: {<DT>?<JJ>*<NN>}"
#     grammer = "NP: {<DT>?<JJ>*<NN.*>}"
    cp = nltk.RegexpParser(grammer)
    new_CRE=[]
    for i in range(0,len(CRE)):
      # output=nltk.chunk.ne_chunk(CRE[i])
      output = cp.parse(CRE[i])
      new_CRE.append(output)
    new_HA=[]
    for i in range(0,len(HA)):
      # output=nltk.chunk.ne_chunk(HA[i])
      if(not HA[i]):
        continue
      output = cp.parse(HA[i])
      new_HA.append(output)
    #extracting noun phrases from CRE
    CRE_NP = []
    for i in range(0,len(new_CRE)):
      tree = new_CRE[i]
      for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
          if(subtree.label()=="NP"):
            np=[]
            for leaf in subtree:
              np.append(leaf[0])
            # np=np[:-1]
            CRE_NP.append(np)
    # print(CRE_NP)
    #extracting noun phrases from HA
    HA_NP = []
    for i in range(0,len(new_HA)):
      tree = new_HA[i]
      for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
          if(subtree.label()=="NP"):
            np=[]
            for leaf in subtree:
              np.append(leaf[0])
            # np=np[:-1]
            HA_NP.append(np)
    # print(HA_NP)
    def intersection(lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3
    def remove_duplication(lst1):
        lst2=[]
        for i in lst1:
          if(i not in lst2): lst2.append(i)
        return lst2
    def union_list(lst1,lst2):
        lst3 = lst1+lst2
        return lst3
    CGT = intersection(CRE_NP,HA_NP)
    # print(CGT)
    new_CRE_tokens1 = []
    cgt_set = set()
    for i in CGT: 
      for j in i:
        cgt_set.add(j)

    for i in new_CRE_tokens:
      tmp=[]
      for j in i:
        if(j in cgt_set):tmp.append("_"+j+"_")
        else :tmp.append(j)
      new_CRE_tokens1.append(tmp)
    # print(new_CRE_tokens1)
    from gensim.models import Word2Vec
    gensim_corpus = union_list( new_CRE_tokens1, new_HA_tokens)
    model = Word2Vec(min_count=1, vector_size=100, window=10,sg=1,negative=15)
    model.build_vocab(gensim_corpus)
    model.train(gensim_corpus, total_examples = model.corpus_count, epochs=model.epochs)
    # str=CGT[0]
    # model.wv.most_similar(str)
    final_CGT = []
    for i in CGT:
      # sum=model.wv.similarity(i,"_"+i+"_")
      # if(sum>=0.5):
      sum=0.00
      cnt=0.00
      chk=1
      for j in i:
        if model.wv.similarity(j,"_"+j+"_")>=0.5:
          chk=0
    # sum+=model.wv.similarity(j,"_"+j+"_")
    # cnt+=1
      if chk==0:
        final_CGT.append(i)
    # print(final_CGT)
    final_result = []
    for i in final_CGT:
      str=""
      for j in i:
        str+=j+" "
      str = str[:-1]
      final_result.append(str)
    final_result = remove_duplication(final_result)
    # print(len(final_result))
    final_result.sort()
    df = pd.DataFrame(final_result)
    return df

In [ ]:
from flask import Flask, render_template, request,make_response
import io
import csv
import pandas as pd

In [11]:
# from flask.ext import excel
app=Flask(__name__)
# run_with_ngrok(app)
@app.route("/")
def index():
  return render_template('index.html')
@app.route("/ml",methods=['POST','GET'])
def test():
  if request.method=='POST':
    csvFile=request.files['csv']
    df=pd.read_csv(csvFile)
#     df=pd.DataFrame(csvFile)
#     print(df.columns)
    ans=func(df)
    data=ans.to_csv()
    response = make_response(data)
    cd = 'attachment; filename=mycsv.csv'
    response.headers['Content-Disposition'] = cd 
    response.mimetype='text/csv'
    return response
    
#     print(data)
#     return data
    
if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
